In [8]:
import argparse
import os
import random
import torch
import torch.cuda
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation


import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import lib

import os.path

from lib.load_data_cifar10 import load_data_cifar10


print(torch.cuda.is_available())

ngpu = 4
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

latent_size = 200

criterion = torch.nn.BCEWithLogitsLoss()
lr = 1e-5
batch_size = 100
num_epochs = 5

train_loader, test_loader = load_data_cifar10(batch_size,7,0.7)



False
Files already downloaded and verified
Files already downloaded and verified


# Training

In [0]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [0]:
# Training Loop

alpha = 0.9

from lib.model_cifar import Encoder, Generator, Discriminator

from lib.train_setting_cifar10 import train_settings
from lib.train_setting_cifar10 import model_create


netE , netG, netD = model_create(ngpu,latent_size,device)

netE.apply(weights_init)
netG.apply(weights_init)
netD.apply(weights_init)

criterion = torch.nn.BCEWithLogitsLoss()

optimizerG = optim.Adam([{'params' : netE.parameters()},
                        {'params' : netG.parameters()}], lr=lr, betas=(0.5, 0.999))

optimizerD = optim.Adam(netD.parameters(), lr = lr , betas = (0.5,0.999))

In [0]:

try:
    print('j')
    netG.load_state_dict(torch.load('./saved_model/generator_cifar.pk1'))
    print('h')
    netE.load_state_dict(torch.load('./saved_model/encoder_cifar.pk1'))
    netD.load_state_dict(torch.load('./saved_model/discriminator_cifar.pk1'))
    print('h')
    print("\n--------model restored--------\n")
except:
    print("\n--------model not restored--------\n")
    pass

In [11]:
for epoch in range (num_epochs):

  
  loss_D, Loss_R = 0 , 0
  
  netD.train()
  netE.train()
  netG.train()

  for i, data in enumerate(train_loader) :
    if i > 377 :
      break
    else : 
      
      #print(random_noise)
      netD.zero_grad()
      netG.zero_grad()
      netE.zero_grad()


      images, labels = data


      # random_noise = 2*(0.02)*torch.rand(images.shape)-1
      images = 2*images-1 #+random_noise
      images = images.to(device)
  
      E_x = netE.forward(images) #z_gen

      z = 2*torch.rand(batch_size,latent_size) - 1
      z = z.to(device)
      
      Gz = netG.forward(z) #x_gen 
      

      #let's recover the las layer and the output of the discrimintor
      D_E , D_E_F_L = netD.forward(E_x,images)
      D_G , D_G_F_L = netD.forward(z,Gz)
      
      #Loss function
      loss_DE = torch.mean(criterion(torch.ones_like(D_E), D_E))
      loss_DG = torch.mean(criterion(torch.zeros_like(D_G), D_G))
      loss_D = loss_DG + loss_DE

      loss_G = torch.mean(criterion(torch.ones_like(D_G), D_G))
      loss_E = torch.mean(criterion(torch.zeros_like(D_E), D_E))
      
      #loss_D = torch.mean(abs(D_E_F_L-D_G_F_L))
      
      # D_loss.append(loss_D)
      #Score 
      
      A_x = alpha*loss_D + (1-alpha)*loss_G
      
      # print(str(epoch) + '  ' + str(loss_G)+'   '+ str(loss_D)+ '   ' +str(A_x))
      # optimizerD.zero_grad()
      loss_D.backward(retain_graph=True)
      optimizerD.step()

      # optimizerG.zero_grad()
      loss_G.backward()
      loss_E.backward()
      optimizerG.step()

      if i %100 ==0 :
            torch.save(netG.state_dict(),'./saved_model/generator_cifar.pk1')
            torch.save(netD.state_dict(),'./saved_model/discriminator_cifar.pk1' )
            torch.save(netE.state_dict(),'./saved_model/encoder_cifar.pk1' )
            print("{}th iteration gen_loss: {} dis_loss: {} enc_loss: {} anomaly-score :{}".format(i,loss_G.data,loss_D.data, loss_E.data,A_x.data))
    # v_utils.save_image(gen_fake.data[0:25],"./result/gen_{}_{}.png".format(i,j), nrow=5)
   
    # Anomaly_Score.append(A_x)
  

0th iteration gen_loss: 0.8425693511962891 dis_loss: 1.5342867374420166 enc_loss: 0.6931470632553101 anomaly-score :1.465114951133728
100th iteration gen_loss: 0.31807324290275574 dis_loss: 1.0092339515686035 enc_loss: 0.6931470632553101 anomaly-score :0.9401178359985352
200th iteration gen_loss: 0.31552135944366455 dis_loss: 1.0069878101348877 enc_loss: 0.6931470632553101 anomaly-score :0.9378411173820496
300th iteration gen_loss: 0.31477823853492737 dis_loss: 1.0066378116607666 enc_loss: 0.6931470632553101 anomaly-score :0.9374518394470215
0th iteration gen_loss: 0.31468671560287476 dis_loss: 1.0066014528274536 enc_loss: 0.6931470632553101 anomaly-score :0.9374099969863892
100th iteration gen_loss: 0.31435397267341614 dis_loss: 1.0065189599990845 enc_loss: 0.6931470632553101 anomaly-score :0.9373024106025696
200th iteration gen_loss: 0.3141357898712158 dis_loss: 1.0064810514450073 enc_loss: 0.6931470632553101 anomaly-score :0.9372465014457703
300th iteration gen_loss: 0.3139953911304

In [12]:
D_loss = []
EG_loss = []
Anomaly_Score_test =[]
for i, data in enumerate(test_loader) :

  images, labels = data
  images = 2*images-1
  # print(images)
  images = images.to(device)

  # z = 2*torch.rand(batch_size,latent_size) - 1
  # z = z.to(device)

  
  Z_gen = netE.forward(images)
  D_loss.append(Z_gen)
  # plt.imshow(Z_gen)
  G_reco = netG.forward(Z_gen)

  #let's recover the las layer and the output of the discrimintor
  D_E , D_E_F_L = netD(Z_gen,images)
  D_G , D_G_F_L = netD(Z_gen,G_reco)

  #reconstruction loss 
  loss_R = torch.norm(images - G_reco, p = 1)
  loss_D = torch.norm(D_E_F_L-D_G_F_L, p =1)

  # D_loss.append(loss_D)
  #Score 

  A_x = alpha*loss_D + (1-alpha)*loss_R

  # print(str(loss_R)+'   '+ str(loss_D)+ '   ' +str(A_x))
  # print(labels)
  print("{}th iteration gen_loss: {} dis_loss: {} anomaly_score : {} ".format(i,loss_R.data,loss_D.data, A_x.data))
 

  Anomaly_Score_test.append(A_x)
 


0th iteration gen_loss: 139684.171875 dis_loss: 22937.40234375 anomaly_score : 34612.078125 
1th iteration gen_loss: 139620.78125 dis_loss: 22799.859375 anomaly_score : 34481.953125 
2th iteration gen_loss: 142841.15625 dis_loss: 22719.025390625 anomaly_score : 34731.23828125 
3th iteration gen_loss: 140489.109375 dis_loss: 22936.62109375 anomaly_score : 34691.87109375 
4th iteration gen_loss: 149906.78125 dis_loss: 23043.96875 anomaly_score : 35730.25 
5th iteration gen_loss: 144861.421875 dis_loss: 22807.689453125 anomaly_score : 35013.0625 
6th iteration gen_loss: 144361.328125 dis_loss: 22897.732421875 anomaly_score : 35044.09375 
7th iteration gen_loss: 137997.875 dis_loss: 22818.658203125 anomaly_score : 34336.578125 
8th iteration gen_loss: 146608.46875 dis_loss: 22625.68359375 anomaly_score : 35023.9609375 
9th iteration gen_loss: 137509.71875 dis_loss: 23024.412109375 anomaly_score : 34472.94140625 
10th iteration gen_loss: 144934.59375 dis_loss: 22859.87890625 anomaly_score :

# AUC

In [18]:
from lib.data_after import do_prc, is_anomalous, Ano_to_list

label = []
for i, data in enumerate(test_loader) :
   images,labels = data
   label.append(labels)


ano_score = Ano_to_list(Anomaly_Score_test)

true_labels = is_anomalous(label,7)
prc_auc = do_prc(ano_score,true_labels,file_name= 'results_cifar_horse',directory= './results', plot = True)

print('The AUC value for '+ str(prc_auc))

The AUC value for 0.411488469879612
